<a href="https://colab.research.google.com/github/daothuphuong98/COVID19-NER/blob/main/BiLSTM_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19.git

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 23), reused 41 (delta 18), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [ ]:
!pip install seqeval tensorflow_addons gensim==4.1.2 git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-cdcl4lia
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-cdcl4lia
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 9.0 MB/s 
     |████████████████████████████████| 24.1 MB 104.0 MB/s 
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101076 sha256=5a21ec1c6976532a39f11ed020177da7aea478c248ac4e48c0a35777b85a9ce1
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_m767eg/wheels/67/d2/f4/96ae3c3c62d1e05abfc8860ad0c1207794726d44ebbbb547f3
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=35cf1ba6cb5a11f1a2abbf5b4314f532016d8f03711dc03425a98d170ad33b2a
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843

In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Input, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers, Sequential
from seqeval.metrics import f1_score, classification_report
from keras_contrib.layers import CRF
import tensorflow.keras.optimizers as Optimizer
import plotly.express as px
from gensim.models import FastText
pd.set_option('display.max_colwidth', -1)

<ipython-input-25-69eaf9fad717>:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


#Data

In [ ]:
def handle_file(link):
    with open(link) as f:
        contents = f.readlines()
    idx_sentence, idx_arr, word, tag = 0, [], [], []
    for line in contents:
        arr_info = line.split(' ')
        # print(arr_info)
        if len(arr_info) == 1:
            idx_sentence += 1
            continue
        word.append(arr_info[0])
        tag.append(arr_info[1].replace('\n', ''))
        idx_arr.append(idx_sentence)
    return pd.DataFrame(
        data = {
            'idx': idx_arr,
            'word': word,
            'tag': tag
        }
    )

In [ ]:
word_train_data = handle_file('/content/PhoNER_COVID19/data/word/train_word.conll')
word_test_data = handle_file('/content/PhoNER_COVID19/data/word/test_word.conll')
word_val_data = handle_file('/content/PhoNER_COVID19/data/word/dev_word.conll')

word_train_data['word_process'] = word_train_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_test_data['word_process'] = word_test_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())
word_val_data['word_process'] = word_val_data['word'].apply(lambda x: re.sub('[0-9]+', "<NUM>", x).lower())

filter_string = '?!"/\{}().,'
word_train_data = word_train_data[word_train_data['word_process'].apply(lambda x: x not in filter_string)]
word_test_data = word_test_data[word_test_data['word_process'].apply(lambda x: x not in filter_string)]
word_val_data = word_val_data[word_val_data['word_process'].apply(lambda x: x not in filter_string)]

word_train_data_group = word_train_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 
word_test_data_group = word_test_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 
word_val_data_group = word_val_data.groupby('idx').agg({'word_process': ' '.join, 'word': ' '.join, 'tag': list}) 

word_train_data_group['len_tag'] = word_train_data_group['tag'].apply(lambda x: len(x))
word_val_data_group['len_tag'] = word_val_data_group['tag'].apply(lambda x: len(x))
word_test_data_group['len_tag'] = word_test_data_group['tag'].apply(lambda x: len(x))

# train_data_group = train_data_group[train_data_group['len_tag'] <= 55].reset_index()
# val_data_group = val_data_group[val_data_group['len_tag'] <= 55].reset_index()
# test_data_group = test_data_group[test_data_group['len_tag'] <= 55].reset_index()

In [ ]:
word_train_data_group[word_train_data_group['len_tag'] < 15].sample(20)

,word_process,word,tag,len_tag
idx,,,,
1966,tại hà_nội bệnh_nhân đi làm và gặp_gỡ bạn_bè tại nhiều nơi,Tại Hà_Nội bệnh_nhân đi làm và gặp_gỡ bạn_bè tại nhiều nơi,"[O, B-LOCATION, O, O, O, O, O, O, O, O, O]",11
4004,trước đó ngày <num>/<num> nữ doanh_nhân đi từ việt_nam sang mỹ có quá_cảnh hàn_quốc,Trước đó ngày 22/2 nữ doanh_nhân đi từ Việt_Nam sang Mỹ có quá_cảnh Hàn_Quốc,"[O, O, O, B-DATE, O, O, O, O, B-LOCATION, O, B-LOCATION, O, O, B-LOCATION]",14
3010,ngày <num>/<num> bệnh_nhân ra hà_nội,Ngày 25/7 bệnh_nhân ra Hà_Nội,"[O, B-DATE, O, O, B-LOCATION]",5
403,chồng của bà đã tử_vong do covid - <num> và suy thận mạn,Chồng của bà đã tử_vong do Covid - 19 và suy thận mạn,"[O, O, O, O, O, O, O, O, O, O, B-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE, I-SYMPTOM_AND_DISEASE]",13
913,bé trai <num> tuổi được ra viện sáng nay ngụ phường cát_lái quận <num>,Bé trai 10 tuổi được ra viện sáng nay ngụ phường Cát_Lái quận 2,"[O, O, O, O, O, O, O, O, O, O, B-LOCATION, I-LOCATION, B-LOCATION, I-LOCATION]",14
845,cô từ mỹ_về quá_cảnh ở philippines nhập_cảnh việt_nam ngày <num> - <num>,Cô từ Mỹ_về quá_cảnh ở Philippines nhập_cảnh Việt_Nam ngày 13 - 3,"[O, O, B-LOCATION, O, O, B-LOCATION, O, B-LOCATION, O, B-DATE, I-DATE, I-DATE]",12
958,theo đó ca bệnh <num> bệnh_nhân <num> là nam <num> tuổi tân_bình thành_phố hải_dương,Theo đó ca bệnh 963 bệnh_nhân 963 là nam 30 tuổi Tân_Bình thành_phố Hải_Dương,"[O, O, O, O, B-PATIENT_ID, O, B-PATIENT_ID, O, B-GENDER, B-AGE, O, B-LOCATION, B-LOCATION, I-LOCATION]",14
3420,ngày <num>/<num> anh này đi chuyến ak<num> từ malaysia tới tp hcm,Ngày 6/3 anh này đi chuyến AK1502 từ Malaysia tới TP HCM,"[O, B-DATE, O, O, O, O, B-TRANSPORTATION, O, B-LOCATION, O, B-LOCATION, I-LOCATION]",12
2123,ông là người thứ <num> ở vĩnh_phúc dương_tính ncov,Ông là người thứ 11 ở Vĩnh_Phúc dương_tính nCoV,"[O, O, O, O, O, O, B-LOCATION, O, O]",9


#CRF

In [ ]:
class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """    
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim) 
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)

# FastText

In [ ]:
word_trained = False
syll_trained = False

In [ ]:
def train_model_fasttext(train_data, size=100, window=5, min_count=3, negative=5, sg=1, alpha=0.01, epoch=50):
    train_data = [each.split() for each in train_data]
    model_fasttext = FastText(vector_size=size, window=window, min_count=min_count, 
                              workers=4, sg=1, negative=negative, alpha=alpha)
    model_fasttext.build_vocab(train_data)
    model_fasttext.train(train_data, total_examples=model_fasttext.corpus_count, epochs=epoch)
    return model_fasttext

In [ ]:
if word_trained:
    word_md = FastText.load('word_model_fasttext_gensim.bin')
else:
    word_md = train_model_fasttext([*word_train_data_group['word_process'], 
                                    *word_val_data_group['word_process']], 
                                    epoch=50, window=5)
    word_md.save('word_model_fasttext_gensim.bin')

In [ ]:
word_tokenizer = Tokenizer(num_words=len(word_md.wv.key_to_index), lower=True, filters="", oov_token='-OOV-')
word_tokenizer.fit_on_texts([*word_train_data_group['word_process'], *word_val_data_group['word_process']])
word_index = word_tokenizer.word_index
emb_mean, emb_std = -0.5,0.5
embed_size = 100  
nb_words = len(word_index) + 1
word_embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= nb_words: 
        continue
    if word in word_md.wv.key_to_index:
        word_embedding_matrix[i] = word_md.wv.get_vector(word)

In [ ]:
word_max_length = 60
word_X_train = word_tokenizer.texts_to_sequences(word_train_data_group['word_process'])
word_X_val = word_tokenizer.texts_to_sequences(word_val_data_group['word_process'])
word_X_test = word_tokenizer.texts_to_sequences(word_test_data_group['word_process'])

word_X_train = pad_sequences(word_X_train, maxlen=word_max_length, padding='post')
word_X_val = pad_sequences(word_X_val, maxlen=word_max_length, padding='post')
word_X_test = pad_sequences(word_X_test, maxlen=word_max_length, padding='post')

In [ ]:
word_tag_tokenizer = Tokenizer(filters="", lower=False)
word_tag_tokenizer.fit_on_texts(word_train_data_group['tag'].apply(lambda x: ' '.join(x)))
word_tag_index = word_tag_tokenizer.word_index
word_tag_size = len(word_tag_index)+1

word_y_train = word_tag_tokenizer.texts_to_sequences(word_train_data_group['tag'].apply(lambda x: ' '.join(x)))
word_y_val = word_tag_tokenizer.texts_to_sequences(word_val_data_group['tag'].apply(lambda x: ' '.join(x)))
word_y_test = word_tag_tokenizer.texts_to_sequences(word_test_data_group['tag'].apply(lambda x: ' '.join(x)))

word_y_train = pad_sequences(word_y_train, maxlen=word_max_length, padding='post')
word_y_val = pad_sequences(word_y_val, maxlen=word_max_length, padding='post')
word_y_test = pad_sequences(word_y_test, maxlen=word_max_length, padding='post')

In [ ]:
word_y_train = np.asarray([to_categorical(i, num_classes=word_tag_size) for i in word_y_train])
word_y_val = np.asarray([to_categorical(i, num_classes=word_tag_size) for i in word_y_val])
word_y_test = np.asarray([to_categorical(i, num_classes=word_tag_size) for i in word_y_test])

#MODEL

## Without CRF

In [ ]:
def create_model(embeddings_matrix, vocab_size, embedding_dim, max_length):
    input = Input(shape = (max_length, ), dtype='int32', name='input_text')
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                  weights=[word_embedding_matrix])(input)
    x = Bidirectional(LSTM(units=max_length, return_sequences=True, 
                                recurrent_dropout=0.01))(x)                        
    x = TimeDistributed(Dense(128, activation='relu', kernel_initializer='he_normal'))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x) 
    output = Dense(word_tag_size, activation='softmax', kernel_initializer='he_normal')(x)
    model_final = Model(input, output)
    model_final.compile(optimizer=Optimizer.Adam(lr=0.005), loss='categorical_crossentropy',
                        metrics=['accuracy'])
    return model_final

In [ ]:
model = create_model(word_embedding_matrix, nb_words, embed_size, word_max_length)
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_text (InputLayer)     [(None, 60)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 60, 100)           451700    
                                                                 
 bidirectional_2 (Bidirectio  (None, 60, 120)          77280     
 nal)                                                            
                                                                 
 time_distributed_2 (TimeDis  (None, 60, 128)          15488     
 tributed)                                                       
                                                                 
 batch_normalization_4 (Batc  (None, 60, 128)          512       
 hNormalization)                                                 
                                                           

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(word_X_train, word_y_train, batch_size=64, epochs=10, validation_data = (word_X_val, word_y_val), verbose=1)

Epoch 1/10
79/79 [==============================] - 38s 423ms/step - loss: 0.3274 - accuracy: 0.9184 - val_loss: 0.2778 - val_accuracy: 0.9215
Epoch 2/10
79/79 [==============================] - 35s 440ms/step - loss: 0.0750 - accuracy: 0.9779 - val_loss: 0.1517 - val_accuracy: 0.9633
Epoch 3/10
79/79 [==============================] - 34s 425ms/step - loss: 0.0500 - accuracy: 0.9852 - val_loss: 0.0889 - val_accuracy: 0.9763
Epoch 4/10
79/79 [==============================] - 34s 426ms/step - loss: 0.0373 - accuracy: 0.9893 - val_loss: 0.0851 - val_accuracy: 0.9765
Epoch 5/10
79/79 [==============================] - 33s 417ms/step - loss: 0.0303 - accuracy: 0.9912 - val_loss: 0.0777 - val_accuracy: 0.9796
Epoch 6/10
79/79 [==============================] - 33s 420ms/step - loss: 0.0264 - accuracy: 0.9923 - val_loss: 0.0827 - val_accuracy: 0.9796
Epoch 7/10
79/79 [==============================] - 33s 418ms/step - loss: 0.0229 - accuracy: 0.9933 - val_loss: 0.0739 - val_accuracy: 0.9822

In [ ]:
def get_tags(sequences, tag_index):
    sequence_tags = []
    for sequence in sequences:
        sequence_tag = []
        for categorical in sequence:
            sequence_tag.append(tag_index.get(np.argmax(categorical)))
        sequence_tags.append(sequence_tag)
    return sequence_tags

def predict(model, tag_tokenizer, sent):
    tag_index = tag_tokenizer.word_index
    tag_size = len(tag_index) + 1
    pred = model.predict(sent)
    sequence_tags = get_tags(pred, {i: t for t, i in tag_index.items()})
    for idx, each in enumerate(sequence_tags):
        try:
           idx_cut = each.index(None)
        except:
           idx_cut = len(each) + 1
        sequence_tags[idx] = each[:idx_cut]
    return sequence_tags

In [ ]:
res = predict(model, word_tag_tokenizer, word_X_test)
print(classification_report(word_test_data_group['tag'].apply(lambda x: x[:word_max_length]), res))

94/94 [==============================] - 4s 36ms/step
                     precision    recall  f1-score   support

                AGE       0.86      0.90      0.88       572
               DATE       0.95      0.95      0.95      1644
             GENDER       0.87      0.87      0.87       453
                JOB       0.48      0.53      0.50       173
           LOCATION       0.81      0.84      0.83      4413
               NAME       0.74      0.70      0.72       314
       ORGANIZATION       0.72      0.70      0.71       769
         PATIENT_ID       0.94      0.93      0.93      1976
SYMPTOM_AND_DISEASE       0.68      0.68      0.68      1128
     TRANSPORTATION       0.90      0.88      0.89       193

          micro avg       0.83      0.84      0.84     11635
          macro avg       0.80      0.80      0.80     11635
       weighted avg       0.83      0.84      0.84     11635



## With CRF

In [ ]:
def create_model_crf(embeddings_matrix, vocab_size, embedding_dim, max_length):
    input = Input(shape = (max_length, ), dtype='int32', name='input_text')
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                  weights=[word_embedding_matrix])(input)
    x = Bidirectional(LSTM(units=max_length, return_sequences=True, 
                                recurrent_dropout=0.01))(x)                        
    x = TimeDistributed(Dense(128, activation='relu', kernel_initializer='he_normal'))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x) 
    x = Dense(word_tag_size, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.1)(x)   
    crf = CRF(word_tag_size)
    output = crf(x)
    model_final = Model(input, output)
    model_final.compile(optimizer=Optimizer.Adam(lr=0.005), loss=crf.loss,
                        metrics=[crf.accuracy])

    return model_final

In [ ]:
model = create_model_crf(word_embedding_matrix, nb_words, embed_size, word_max_length)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_text (InputLayer)     [(None, 60)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 60, 100)           451700    
                                                                 
 bidirectional_1 (Bidirectio  (None, 60, 120)          77280     
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 60, 128)          15488     
 tributed)                                                       
                                                                 
 batch_normalization_2 (Batc  (None, 60, 128)          512       
 hNormalization)                                                 
                                                             

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(word_X_train, word_y_train, batch_size=64, epochs=10, validation_data = (word_X_val, word_y_val), verbose=1)

Epoch 1/10
79/79 [==============================] - 57s 586ms/step - loss: 98.6042 - viterbi_accuracy: 0.7059 - val_loss: 104.9873 - val_viterbi_accuracy: 0.9095
Epoch 2/10
79/79 [==============================] - 46s 580ms/step - loss: 30.5124 - viterbi_accuracy: 0.9715 - val_loss: 79.2038 - val_viterbi_accuracy: 0.9248
Epoch 3/10
79/79 [==============================] - 45s 565ms/step - loss: 12.7847 - viterbi_accuracy: 0.9783 - val_loss: 65.7644 - val_viterbi_accuracy: 0.9314
Epoch 4/10
79/79 [==============================] - 44s 564ms/step - loss: 7.9843 - viterbi_accuracy: 0.9818 - val_loss: 56.8827 - val_viterbi_accuracy: 0.9335
Epoch 5/10
79/79 [==============================] - 45s 570ms/step - loss: 5.7889 - viterbi_accuracy: 0.9847 - val_loss: 50.7504 - val_viterbi_accuracy: 0.9361
Epoch 6/10
79/79 [==============================] - 46s 582ms/step - loss: 4.6077 - viterbi_accuracy: 0.9866 - val_loss: 46.5521 - val_viterbi_accuracy: 0.9348
Epoch 7/10
79/79 [==================

In [ ]:
def get_tags(sequences, tag_index):
    sequence_tags = []
    for sequence in sequences:
        sequence_tag = []
        for categorical in sequence:
            sequence_tag.append(tag_index.get(np.argmax(categorical)))
        sequence_tags.append(sequence_tag)
    return sequence_tags

def predict(model, tag_tokenizer, sent):
    tag_index = tag_tokenizer.word_index
    tag_size = len(tag_index) + 1
    pred = model.predict(sent)
    sequence_tags = get_tags(pred, {i: t for t, i in tag_index.items()})
    for idx, each in enumerate(sequence_tags):
        try:
           idx_cut = each.index(None)
        except:
           idx_cut = len(each) + 1
        sequence_tags[idx] = each[:idx_cut]
    return sequence_tags

In [ ]:
res = predict(model, word_tag_tokenizer, word_X_test)
print(classification_report(word_test_data_group['tag'].apply(lambda x: x[:word_max_length]), res))

                     precision    recall  f1-score   support

                AGE       0.91      0.88      0.90       572
               DATE       0.97      0.96      0.96      1644
             GENDER       0.90      0.88      0.89       453
                JOB       0.67      0.36      0.47       173
           LOCATION       0.87      0.85      0.86      4413
               NAME       0.89      0.59      0.71       314
       ORGANIZATION       0.86      0.73      0.79       769
         PATIENT_ID       0.94      0.93      0.93      1976
SYMPTOM_AND_DISEASE       0.79      0.66      0.72      1128
     TRANSPORTATION       0.97      0.81      0.88       193

          micro avg       0.89      0.84      0.86     11635
          macro avg       0.88      0.77      0.81     11635
       weighted avg       0.89      0.84      0.86     11635

